In [1]:
from transformers import BartTokenizer, BartForConditionalGeneration

In [2]:

def load_distilbert_model(model_name="facebook/bart-large"):
    tokenizer = BartTokenizer.from_pretrained(model_name)
    model = BartForConditionalGeneration.from_pretrained(model_name)
    return tokenizer, model


In [3]:
def summarize_text(tokenizer, model, text, max_chunk_length, summary_max_length):
    inputs = tokenizer(text, return_tensors="pt", max_length=max_chunk_length, truncation=True)
    summary_ids = model.generate(inputs["input_ids"], max_length=summary_max_length, min_length=200, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [4]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
def loading_data(url):
    loader=UnstructuredURLLoader(urls=[
    url,
    ])
    data=loader.load()
    return data    

In [6]:
def to_chunks(data):
    text_splitter=RecursiveCharacterTextSplitter(
        chunk_size=3000,
        chunk_overlap=50
    )
    docs=text_splitter.split_documents(data)
    return docs

In [7]:
def summarize_article(url, model_name="sshleifer/distilbart-cnn-12-6"):
    data = loading_data(url)
    chunks = to_chunks(data)
    tokenizer, model = load_distilbert_model(model_name)
    summaries = []
    for chunk in chunks:
        chunk_text = chunk.page_content
        summary = summarize_text(tokenizer, model, chunk_text,3000,800)
        summaries.append(summary)
    concatenated_summaries = " ".join(summaries)
    #  Second summarization pass: Summarize the concatenated summaries
    intermediate_chunks = [concatenated_summaries[i:i+3000] for i in range(0, len(concatenated_summaries), 3000)]
    final_summaries = []
    for intermediate_chunk in intermediate_chunks:
        final_summary = summarize_text(tokenizer, model, intermediate_chunk,3000,800)
        final_summaries.append(final_summary)
    
    final_summary_text = " ".join(final_summaries)
    
    return final_summary_text

In [8]:
url = "https://www.bbc.com/news/articles/cleezyxjv4jo"
summary = summarize_article(url)
print(summary)

 Deal was blocked earlier this year by a judge in Delaware over concerns it was unfair to shareholders. Deal is worth an estimated 300 times what the top-earning boss in the US made last year. Mr Musk had campaigned fiercely for the payout, which is worth up to $56billion (£43.9bn) The compensation plan gives Mr Musk rights to roughly 300 million shares - the equivalent to a 10% stake in the firm - as a reward for Tesla meeting a number of goals set out in 2018 which are linked to sales, profits and the share price. The vote is not binding and legal experts say it is not clear if the court that blocked the deal will accept the re-vote and allow the company to restore the pay package. The firm's legal headquarters will now be moved to Texas. It has not yet been announced. GameStop raised $2billion in the first half of the company's annual $4bn annual profit run for the first time since 2008. Decision comes as an investigation continues into what the EU calls China’s ‘unfairly
